SMT version for the MCP problem given in the Combinatorial Decision Making and Optimization course.

The model is based on the one already developed for the CP version of the problem with the necessary modifications to make it work for the SMT version.

Necessary libraries:

In [ ]:
#!pip install z3-solver

Necessary imports:

In [16]:
from z3 import * # The Z3 Theorem Prover
import numpy as np # Numpy for matrix operations
import matplotlib.pyplot as plt # Matplotlib for plotting
import time
import json

The variable instances (like number of couriers) are defined in a .dat file. The file is read and the variables are defined.

In [20]:
def printJson():

    solverName = 'Z3'
    numInstances = 1
    formatted_list = ["%.2d" % i for i in range(1, numInstances + 1)]
    simple_list = [i for i in range(1, numInstances + 1)]
   


    for i in range(numInstances):
         # open the file in Instances folder
        f = open(f"Instances/inst{formatted_list[i]}.dat", "r")
        # the first line is the number of couriers
        m = int(f.readline())
        # the second line is the number of items
        n = int(f.readline())
        # the third line is the load size of each courier
        load_size = [int(x) for x in f.readline().split()]
        # the fourth line is the size of each item
        item_size = [int(x) for x in f.readline().split()]
        # the rest is the distance matrix
        distance = []
        for j in range(n+1):
            distance.append([int(x) for x in f.readline().split()])
        # close the file
        f.close()

        print("Solving Instance", i+1, "with Z3")
        obj, sol, time = solve_courier_problem(m,n, load_size, item_size, distance)
        opt = True

        #adding 1 to all the elements of sol
        for j in range(len(sol)):
            sol[j] = np.array(sol[j]) + 1
            sol[j] = sol[j].tolist()



        data = dict([(solverName,dict([
            ('time', time),
            ('optimal',opt),
            ('obj', obj),
            ('sol', sol)
            ]))])


        json_string = json.dumps(data)

        with open(f'./output_folder/SMT/{simple_list[i]}.json', 'w') as outfile:
            outfile.write(json_string)


In [21]:
def solve_courier_problem(m,n, load_size, item_size, distance):

    max_path_length = n-(m-1)
    paths = [[Int("p_%s_%s" % (i,j)) for j in range(max_path_length)] for i in range(m)] 
    # create a matrix of mxn boolean variables for the assignment of items to couriers
    assignment = [[Bool("a_%s_%s" % (i,j)) for j in range(n)] for i in range(m)]
    # Create a solver instance
    s = Solver()

    constraints = []
    constraints.append(n>=m)

    # Main constraints:

    # Each item has a size > 0 and <= max(load size)
    for i in range(n):
        constraints.append(And(item_size[i] > 0, item_size[i] <= max(load_size)))

    # Each item is delivered at most by one courier, and at least by one courier
    for i in range(n):
        constraints.append(Sum([If(paths[c][j] == i, 1, 0) for c in range(m) for j in range(max_path_length)]) == 1)

    # assignement constraints 
    for c in range(m):
        for i in range(n):
            constraints.append(assignment[c][i] == Or([paths[c][j] == i for j in range(max_path_length)]))

    # Each courier can carry at most its load size
    for c in range(m):
        constraints.append(Sum([If(assignment[c][i], item_size[i], 0) for i in range(n)]) <= load_size[c])

    # Each courier must deliver at least one item
    for c in range(m):
        constraints.append(Sum([If(assignment[c][i], 1, 0) for i in range(n)]) >= 1)

    best_max_distance = math.inf
    s.add(constraints)
    courier_distances = [[0] for c in range(m)]
    courier_loads = [[0] for c in range(m)]
    best_courier_distances = [[0] for c in range(m)]
    break_counter = 0
    loop_counter = 0
    st = time.time()
    for l in range(1000):
        if s.check() == sat:
            loop_counter += 1
            model = s.model()
            # get the values of the paths
            paths_values = [[model[paths[i][j]] for j in range(max_path_length)] for i in range(m)]
            # get path for each courier as a list of items, taking only the values in the range [0,n-1]
            paths_items = [[paths_values[i][j].as_long() for j in range(max_path_length) if paths_values[i][j].as_long() < n] for i in range(m)]
            paths_items = [list(filter(lambda x: x != -1, paths_items[i])) for i in range(m)] 
            # get the total distance for each courier by adding also the distance from the depot to the first item and from the last item to the depot
            for c in range(m):
                dist = distance[n][paths_items[c][0]] + distance[paths_items[c][-1]][n]
                for i in range(len(paths_items[c])-1):
                    dist += distance[paths_items[c][i]][paths_items[c][i+1]]
                courier_distances[c][0] = dist

            # only accept solutions with max distance < best_max_distance
            # and update best_max_distance
            max_distance = max([courier_distances[c][0] for c in range(m)])
            if max_distance < best_max_distance:
                best_max_distance = max_distance
                break_counter = loop_counter
                best_courier_distances = courier_distances.copy()

                best_paths_items = paths_items.copy()
                best_courier_distances = courier_distances.copy()
                # print("max_distance:", max_distance)
                # print("paths_items:", paths_items)
                # print("courier_distances:", courier_distances)

            else:
                if loop_counter - break_counter > 200:
                    break
        else:
            break
    et = time.time()
    elapsed_time = int(et - st)
    print(best_paths_items)
    return best_max_distance, best_paths_items, elapsed_time



In [22]:
printJson()

Solving Instance 1 with Z3
[[0, 2, 3], [1, 4, 5]]
Solving Instance 2 with Z3
[[6, 4], [1], [0], [3], [7, 5], [2, 8]]
Solving Instance 3 with Z3
[[4, 3, 1], [5, 2], [6, 0]]
Solving Instance 4 with Z3
[[5], [7], [2], [8], [4, 9], [3], [0], [6, 1]]
Solving Instance 5 with Z3
[[1], [0, 2]]
Solving Instance 6 with Z3
[[2], [4], [5], [0, 7, 3], [6], [1]]
Solving Instance 7 with Z3
